In [14]:
import numpy as np

def RELU(x):
    return np.maximum(0,x)

def relu_der(z):
    return 1.*(z>0)

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

def softmax_der(x):
    prob = softmax(x)
#     print(np.shape(prob))
    r = np.matmul(prob.T, (np.ones_like(prob)-prob))
#     print(np.shape(r))
    return r

def oneHotEncode(y,softmaxClasses):
        yactual = np.matrix(np.zeros(softmaxClasses)).T
        yactual[y] = 1
        return yactual    
    
def crossEntropy(ypred,y,softmaxClasses):
    """Compute crossentropy from logits[batch,n_classes] and ids of correct answers"""
    #one hot encode
    yactual = oneHotEncode(y,softmaxClasses)
    l = np.log(ypred)
    return -np.sum(l*yactual.T)
#     logits_for_answers = logits[np.arange(len(logits)),reference_answers]
#     xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))
#     return xentropy
    

def delta_cross_entropy(y, X):
          m = y.shape[0]
          grad = softmax(X)
          idx = np.where(grad == y*grad)
          grad[idx] = grad[idx] - 1
          grad = grad/m
          return grad
def calcLoss(ypred, yactual):
    if self.softmax == True:
        return crossEntropy(ypred,yactual, self.softmaxClasses)

In [20]:
class neuralNetwork():    
    FCreluLayers = 0  
    Weights = []
    Biases = []
    softmax = False
    softmaxClasses = 0
    loss = []
    delta = [[]];    dW = [[]];    db = [[]]
    act = []
    logits = [[]]
    def FullyConnectedreluLayers(self, values):
        assert len(values) == self.FCreluLayers+1, "Number of fully connected RELU layers are not the same."
        self.Weights.append([])
        self.Biases.append([])
        for i in range(1, self.FCreluLayers+1):            
            self.Weights.append(np.random.normal(0, 0.1, ([values[i-1], values[i]])))
            self.Biases.append(np.random.normal(0, 0.1, ([values[i], 1])))
            self.delta.append([]);    self.dW.append([]);    self.db.append([])
            print(np.shape(self.Weights[i]))   
    
    def softmaxLayer(self):
        previous = self.Weights[-1].shape[1]
        self.Weights.append(np.random.normal(0,0.1,[previous, self.softmaxClasses]))
        self.Biases.append(np.random.normal(0, 0.1, (self.softmaxClasses, 1)))
        self.delta.append([]);    self.dW.append([]);    self.db.append([])
        
    def calcLoss(self,ypred, yactual):
        if self.softmax == True:
            return crossEntropy(ypred,yactual, self.softmaxClasses)
    
    def feedForward(self,X):
#         a = []
        self.act.append(X.T)
#         l = self.FCLayers if softmax == False else self.FCLayers-1
        
        for i in range(1, self.FCreluLayers+1):
#             print(" weights shape: ", np.shape(self.Weights[i].T) , "a shape: ", np.shape(self.act[i-1]))
            self.logits.append(np.matmul(self.Weights[i].T, self.act[i-1]) + self.Biases[i])
            self.act.append( RELU(self.logits[-1]) )
            
#         print("-----")
        if self.softmax == True:
            self.logits.append(np.matmul(self.Weights[-1].T, self.act[-1]) + self.Biases[-1])
            self.act.append(softmax(self.logits[-1]))
#             print(np.sum(self.act[-1]))   
#             print(self.act[-1])
        print("act shapes: ", [np.shape(self.act[i]) for i in range(len(self.act))])
        return self.act[-1]
    
    def backprop(self, ypred, yact):
        
        L = len(self.delta)-1       
        for i in range(L,0,-1):     
            self.delta[i] = delta_cross_entropy(ypred,yact)
            self.db[i] = self.delta[i].copy()
            self.dW[i] = self.act[i-1] * self.delta[i]
            if i > 1:
                if i == L:
                    xs = np.matmul(self.Weights[i], self.delta[i].T)
                    self.delta[i-1] = np.multiply(softmax_der(self.logits[i-1]), xs.T)
                else:
                    xs = np.matmul(self.Weights[i], self.delta[i].T)
                    self.delta[i-1] = np.multiply(relu_der(self.logits[i-1]), xs.T)
        alpha = 0.001
        for l in range(1,len(self.Weights)):            
            self.Weights[l] = self.Weights[l] - alpha * self.dW[l]
            self.Biases[l] = self.Biases[l] - alpha * self.db[l]           

mnist = neuralNetwork()
mnist.__class__.FCreluLayers = 2 #exclusing the softmax
mnist.__class__.softmax = True
mnist.__class__.softmaxClasses = 6
N = 64
h1 = 24
h2 = 16
mnist.FullyConnectedreluLayers([N,h1,h2])
mnist.softmaxLayer()

ypred = mnist.feedForward( np.matrix(np.ones(64)) )
# print(np.shape(ypred), np.zeros(10))
print("loss: ", mnist.calcLoss(ypred, 2))
mnist.backprop(ypred, np.matrix(np.zeros(6)) )

(64, 24)
(24, 16)
act shapes:  [(64, 1), (24, 1), (16, 1), (6, 1)]
loss:  10.805677198423828


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6 is different from 16)

In [199]:
# from sklearn.datasets import load_digits
# digits = load_digits()
# print("listing digits: ", list(digits))
# print("Number of samples: ", digits.data.shape[0])
# print("Size of each sample: ", digits.data.shape[1])
# print("Listing target names: ", list(digits.target_names))

# y = np.matrix(digits.target).T
# X = np.matrix(digits.data)
# print("Image at index 3: ")
# import matplotlib.pyplot as plt 
# plt.matshow(digits.images[3]) 
# plt.show()

In [106]:

# r = np.array([1,0,0])
# logits = np.array([0.6,0,0.5])
# logits_for_answers = logits[np.arange(len(logits)),r]

In [212]:
np.shape(np.matrix(np.zeros(5)).T)

(5, 1)

In [219]:
def oneHotEncode(y,softmaxClasses):
        yactual = np.matrix(np.zeros(softmaxClasses)).T
        yactual[y] = 1
        return yactual
    
def crossEntropy(ypred,y,softmaxClasses):
    """Compute crossentropy from logits[batch,n_classes] and ids of correct answers"""
    #one hot encode
    yactual = oneHotEncode(y,softmaxClasses)
    print(yactual)
    l = np.log(ypred)
    print(l)
    return - np.sum(l*yactual)/yactual.shape[0]

crossEntropy(np.array([1.6,3.4,1.4,4]), 2, 4)

[[0.]
 [0.]
 [1.]
 [0.]]
[0.47000363 1.22377543 0.33647224 1.38629436]


-0.08411805915530322

In [221]:
def delta_cross_entropy(y, X):
          m = y.shape[0]
          grad = softmax(X)
          idx = np.where(grad == y*grad)
          grad[idx] = grad[idx] - 1
          grad = grad/m
          return grad
delta_cross_entropy(np.array([4,3,2,1]), np.array([3,4,12,4]))

array([ 3.08279633e-05,  8.37990924e-05,  2.49801574e-01, -2.49916201e-01])